#### Домашнее задание по теме «Рекомендации на основе содержания»

* Использовать dataset MovieLens
* Построить рекомендации (регрессия, предсказываем оценку) на фичах:
* TF-IDF на тегах и жанрах
* Средние оценки (+ median, variance, etc.) пользователя и фильма
* Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
links = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\links.csv')
movies = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\movies.csv')
ratings = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\ratings.csv')
tags = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\tags.csv')

In [3]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:
data1 = movies.join(tags.set_index('movieId'), on='movieId')      # movies.merge(tags)
data1 = data1.drop(['userId', 'timestamp'], axis=1).dropna().reset_index(drop=True)
data1.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun
3,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy
4,2,Jumanji (1995),Adventure|Children|Fantasy,magic board game


In [8]:
Tag = {}
for title, group in tqdm(data1.groupby('movieId')):
    Tag[title] = ' '.join(group.tag.unique())
Tag[1]

'pixar fun'

In [9]:
data1['tag'] = data1['movieId'].apply(lambda x: Tag[x].lower())
data1.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar fun
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar fun
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar fun
3,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy magic board game robin williams game
4,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy magic board game robin williams game


In [10]:
data1['genres'] = data1['genres'].apply(lambda x: ' '.join(x.lower().replace(' ', '').replace('-', '').split('|')))
data1.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun
1,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun
2,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun
3,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game
4,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game


In [11]:
Genre = {}
for title, group in tqdm(data1.groupby('movieId')):
    Genre[title] = ' '.join(group.genres.unique())
Genre[1]

'adventure animation children comedy fantasy'

In [12]:
data1['genres'] = data1['movieId'].apply(lambda x: Genre[x])
data1.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun
1,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun
2,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun
3,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game
4,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game


In [13]:
data1 = data1.drop_duplicates(subset=['movieId'], keep='first').reset_index(drop=True)
data1.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun
1,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game
2,3,Grumpier Old Men (1995),comedy romance,moldy old
3,5,Father of the Bride Part II (1995),comedy,pregnancy remake
4,7,Sabrina (1995),comedy romance,remake


In [14]:
data2 = data1.merge(ratings).drop(['timestamp'], axis=1)
data2.head()

,movieId,title,genres,tag,userId,rating
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,1,4.0
1,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,5,4.0
2,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,7,4.5
3,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,15,2.5
4,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,17,4.5


In [15]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48287 entries, 0 to 48286
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  48287 non-null  int64  
 1   title    48287 non-null  object 
 2   genres   48287 non-null  object 
 3   tag      48287 non-null  object 
 4   userId   48287 non-null  int64  
 5   rating   48287 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 2.6+ MB


In [16]:
mean_film = {}   # средняя оценка за фильм
for title, group in tqdm(data2.groupby('movieId')):
    mean_film[title] = group.rating.mean()
    
mean_rating_user = {}  # средняя оценка пользователя
for title, group in tqdm(data2.groupby('userId')):
    mean_rating_user[title] = group.rating.mean()

In [17]:
# количество рейтингов по каждому фильму
num_rating = {}

for i, j in data2.groupby('movieId'):
    num_rating[i] = j.userId.unique().shape[0]
    
# выведем статистику по количеству рейтингов
min_num_rating = np.min([num_rating[i] for i in num_rating.keys()])
max_num_rating = np.max(([num_rating[i] for i in num_rating.keys()]))
mean_num_rating = np.mean(([num_rating[i] for i in num_rating.keys()]))
min_num_rating, max_num_rating, mean_num_rating

(1, 329, 31.072715572715573)

In [18]:
# выведем искомую статистику по каждому фильму (средний рейтинг в соответствии с размерностью рейтинга)
score_film = {}

for i in num_rating.keys():
    score_film[i] = mean_film[i] * (num_rating[i] - min_num_rating) /  (max_num_rating - min_num_rating)
    #score_film[i] = (mean_film[i]  - min_num_rating) /  (max_num_rating - min_num_rating)

In [19]:
# количество кликов по каждому пользователю
num_click = {}

for i, j in data2.groupby('userId'):
    num_click[i] = j.movieId.unique().shape[0]
    
# выведем статистику по количеству кликов пользователя
min_num_click = np.min([num_click[i] for i in num_click.keys()])
max_num_click = np.max(([num_click[i] for i in num_click.keys()]))
mean_num_click = np.mean(([num_click[i] for i in num_click.keys()]))
min_num_click, max_num_click, mean_num_click

(4, 1198, 79.15901639344263)

In [20]:
# выведем искомую статистику по каждому пользователю (в соответствии с размерностью кликов)
score_user = {}

for i in num_click.keys():
    score_user[i] = mean_rating_user[i] * (num_click[i] - min_num_click) /  (max_num_click - min_num_click)
    #score_user[i] = (mean_rating_user[i]  - min_num_click) /  (max_num_click - min_num_click)

In [21]:
data2['mean_rating_film'] = data2['movieId'].apply(lambda x: score_film[x])
data2['mean_rating_user'] = data2['userId'].apply(lambda x: score_user[x])

#data2['mean_rating_film'] = data2['movieId'].apply(lambda x: mean_film[x])
#data2['mean_rating_user'] = data2['userId'].apply(lambda x: mean_rating_user[x])
data2.head()

,movieId,title,genres,tag,userId,rating,mean_rating_film,mean_rating_user
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,1,4.0,2.558168,0.405683
1,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,5,4.0,2.558168,0.109737
2,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,7,4.5,2.558168,0.258228
3,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,15,2.5,2.558168,0.226759
4,1,Toy Story (1995),adventure animation children comedy fantasy,pixar fun,17,4.5,2.558168,0.293872


In [22]:
# модель для каждого пользователя
data_user_1 = data2[data2['userId']==1][['movieId', 'genres', 'tag', 'rating', 'mean_rating_film', 'mean_rating_user']].reset_index(drop=True)
data_user_1.head()

,movieId,genres,tag,rating,mean_rating_film,mean_rating_user
0,1,adventure animation children comedy fantasy,pixar fun,4.0,2.558168,0.405683
1,3,comedy romance,moldy old,4.0,0.506830,0.405683
2,47,mystery thriller,mystery twist ending serial killer,5.0,2.448246,0.405683
3,50,crime mystery thriller,mindfuck suspense thriller tricky twist ending...,5.0,2.622751,0.405683
4,101,adventure comedy crime romance,crime off-beat comedy quirky,5.0,0.253712,0.405683


In [23]:
tfidf1 = TfidfVectorizer()   # for genres
tfidf2 = TfidfVectorizer()   # for tag

In [24]:
tfidf1.fit(data1['genres'])
tfidf2.fit(data1['tag'])


TfidfVectorizer()

In [25]:
data_user_1['genres'] = data_user_1['genres'].apply(lambda x: tfidf1.transform([x]).toarray())
data_user_1['tag'] = data_user_1['tag'].apply(lambda x: tfidf2.transform([x]).toarray())
data_user_1.head()

,movieId,genres,tag,rating,mean_rating_film,mean_rating_user
0,1,"[[0.0, 0.3986132915900188, 0.521641128701684, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",4.0,2.558168,0.405683
1,3,"[[0.0, 0.0, 0.0, 0.0, 0.6431454908530309, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",4.0,0.506830,0.405683
2,47,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",5.0,2.448246,0.405683
3,50,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.5679859772074638,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",5.0,2.622751,0.405683
4,101,"[[0.0, 0.5552693958028675, 0.0, 0.0, 0.3930803...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",5.0,0.253712,0.405683


In [26]:
y = data_user_1['rating'].tolist()

In [27]:
X = data_user_1.drop(['rating'], axis=1)
X.head()

,movieId,genres,tag,mean_rating_film,mean_rating_user
0,1,"[[0.0, 0.3986132915900188, 0.521641128701684, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2.558168,0.405683
1,3,"[[0.0, 0.0, 0.0, 0.0, 0.6431454908530309, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.506830,0.405683
2,47,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2.448246,0.405683
3,50,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.5679859772074638,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2.622751,0.405683
4,101,"[[0.0, 0.5552693958028675, 0.0, 0.0, 0.3930803...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.253712,0.405683


In [46]:
#X['tfidf'] = X.apply(lambda x: np.concatenate((np.array(x['genres']), np.array(x['tag'])), axis=0))
tfidf = {}
for i, j in X.iterrows():
    #tfidf[i] = np.insert(np.hstack((j[0], j[1])), [0,0], [j[2], j[3]])
    tfidf[j[0]] = np.insert(np.hstack((j[1], j[2])), [0,0], [j[3], j[4]])
    #tfidf[i] = np.hstack((j[0], j[1]))
tfidf[1]
#X.head()

array([2.5581679 , 0.40568339, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [29]:
#X['tfidf'] = tfidf.values()
X['tfidf'] = X['movieId'].apply(lambda x: tfidf[x])
X.head()

,movieId,genres,tag,mean_rating_film,mean_rating_user,tfidf
0,1,"[[0.0, 0.3986132915900188, 0.521641128701684, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2.558168,0.405683,"[2.5581678956324447, 0.4056833876987276, 0.0, ..."
1,3,"[[0.0, 0.0, 0.0, 0.0, 0.6431454908530309, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.506830,0.405683,"[0.5068304409005628, 0.4056833876987276, 0.0, ..."
2,47,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2.448246,0.405683,"[2.4482458248227803, 0.4056833876987276, 0.0, ..."
3,50,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.5679859772074638,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2.622751,0.405683,"[2.622750777140124, 0.4056833876987276, 0.0, 0..."
4,101,"[[0.0, 0.5552693958028675, 0.0, 0.0, 0.3930803...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.253712,0.405683,"[0.25371155885471897, 0.4056833876987276, 0.0,..."


In [30]:
XX = np.array(X['tfidf'].tolist())
XX

array([[2.5581679 , 0.40568339, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.50683044, 0.40568339, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [2.44824582, 0.40568339, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [2.02915172, 0.40568339, 0.65216548, ..., 0.        , 0.        ,
        0.        ],
       [0.73190401, 0.40568339, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.4887218 , 0.40568339, 0.54742359, ..., 0.        , 0.        ,
        0.        ]])

In [31]:
XX.shape

(114, 1766)

In [32]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 50)
X_psa = pca.fit_transform(XX)
X_psa

array([[ 1.23665659,  0.47097687, -0.74639783, ...,  0.00168831,
         0.02088308,  0.26388631],
       [-0.75626319, -0.19653345, -0.12346956, ..., -0.29022071,
         0.07054794,  0.12511014],
       [ 1.25781649, -0.32581705, -0.18668961, ..., -0.00825428,
         0.0057453 ,  0.03114525],
       ...,
       [ 0.8640985 ,  0.35630373,  0.23992844, ...,  0.00710473,
        -0.00429788,  0.01185905],
       [-0.50420544, -0.15591879, -0.10721178, ..., -0.01474622,
        -0.06096786, -0.05654314],
       [ 0.34475072,  0.52991356,  0.58739404, ...,  0.01446647,
         0.02880443,  0.02278754]])

In [33]:
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_psa = sc.fit_transform(X_psa)
#X_psa

In [34]:
lr = LinearRegression()

In [35]:
#X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size=0.3)
X_train, X_test, y_train, y_test = train_test_split(X_psa, y, test_size=0.2)

In [36]:
#! pip install scikit-multilearn

In [37]:
#from skmultilearn.model_selection.iterative_stratification import iterative_train_test_split

#X_train, y_train, X_test, y_test = iterative_train_test_split(X_psa, y, test_size=0.3)

In [38]:
lr.fit(X_train, y_train)

LinearRegression()

In [39]:
y_pred = lr.predict(X_test)

In [40]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
        # Судя по результатам Cross-Validation, R^2 score очень сильно различается для разных частей датасета 
        # и в нашем случае постоянно ухудшается,
        # попробуем перетасовать датасет перед тем как делать Cross-Validation

cv = ShuffleSplit(n_splits=15, test_size=0.3)
scores = cross_val_score(lr, X_test, y_test, cv=cv)
scores.mean()

-2.1790494644593545

In [41]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse
#rmse = sqrt(mean_squared_error(y_test, y_pred))

0.9707774715307737

In [42]:
lr.score(X_test, y_test)

-0.8602026406476109

In [43]:
lr.score(X_train, y_train)

0.5695657045301576